# Logistic Regression Model (Predicting Interview Acceptance)

In [29]:
import re
import boto3
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
s3_conn = boto3.client('s3')

import pyspark
from pyspark.sql import SparkSession

sess = sagemaker.Session()

region = boto3.Session().region_name

# S3 bucket where the original mnist data is downloaded and stored.
#downloaded_data_bucket = f"sagemaker-sample-files"
#downloaded_data_prefix = "datasets/image/MNIST"

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
#bucket = sess.default_bucket()
#prefix = "sagemaker/DEMO-linear-mnist"

role = get_execution_role()

Master dataframe imported from S3:

In [30]:
# def max_partition(bucket: str = None,prefix: str =None):
#     response = s3_conn.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter='/')
#     #print(response['CommonPrefixes'])
#     replace_path=prefix+'date_col='
#     print(replace_path)
#     ts=[]
#     for item in response['CommonPrefixes']:
#         #print(item)
#         ts.append(item['Prefix'].replace(replace_path,'').replace('/',''))
#     #print(ts)
#     max_partition=max(ts)
#     print("max parition is", max_partition)
#     s3_path="s3://"+bucket+"/"+prefix+"date_col="+max_partition+"/*"
#     print(s3_path)
#     return s3_path

In [38]:
# max_s3path = max_partition('hiring-curated-bucket', 'master-table/test_csv/')


In [39]:
# master_df_new = spark.read.csv(max_s3path,  header = True, sep=',')
# master_df_new

In [40]:
master_df_new = pd.read_csv(max_s3path)
master_df_new

,hircand_Interview_ID,hircand_Candidate_Name,hircand_Candidate_ICMS_ID,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hircand_Last_Updated,hiremp_Interview_ID,...,sched_timestamp,sched_week_number,sched_slot_hour,sched_modified,sched_last_updated,emp_alias,emp_name,emp_skill,emp_skill_rating,emp_last_updated
0,51092998,Agnese Normanton,7203708,EC2 Systems Engineer,Contract,L9,15:00,Tuesday,2022-02-04T16:18:37Z,51092998,...,2021-12-08T20:21:20Z,4,16:00,Null,2022-04-26T10:54:21Z,ldelepine0,Linea Delepine,Compute and HPC - EC2 (Elastic Compute Cloud),1,2021-12-27T11:35:02Z
1,99241636,Davey Abrams,15918070,Professional Services Consultant,Contract,L8,13:00,Monday,2022-03-20T01:04:55Z,99241636,...,2021-08-18T07:24:37Z,49,12:00,Null,2022-04-24T09:18:34Z,aharral1,Arielle Harral,Database - RDS SQL Server,5,2021-12-29T20:03:24Z
2,64517874,Marrissa Langhor,67232318,EC2 Systems Engineer,Part-time,L6,14:00,Thursday,2022-03-18T12:56:21Z,64517874,...,2021-11-03T01:47:35Z,3,12:00,Null,2021-08-22T12:22:13Z,sdryburgh2,Shayne Dryburgh,ML/AI - Rekognition,1,2021-06-25T07:12:05Z
3,75309134,Aurora Ireland,82904801,Machine Learning Engineer,Internship,L6,12:00,Tuesday,2021-11-26T16:06:23Z,75309134,...,2022-01-06T09:58:41Z,23,10:00,Null,2022-06-14T09:56:02Z,chugonet3,Claudetta Hugonet,Analytics - Lake Formation,3,2021-12-03T04:03:39Z
4,31410407,Debora Greenall,76196522,Cloud Architect,Full-time,L6,16:00,Wednesday,2022-02-20T12:24:35Z,31410407,...,2022-01-31T02:03:52Z,3,14:00,Null,2022-05-29T10:38:42Z,cfawthrop4,Cyrille Fawthrop,ML/AI - Forecast,1,2022-03-20T08:45:19Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,Burgess Baumer,39023047,Delivery Practice Manager,Part-time,L5,12:00,Monday,2022-05-08T22:34:48Z,79716724,...,2021-08-28T08:13:52Z,24,17:00,Null,2021-07-10T19:16:21Z,tvedntyevrn,Terrie Vedntyev,ML/AI - Forecast,5,2022-04-29T00:23:01Z
996,36886778,Jocelyn Howman,40070522,Marketing Manager,Full-time,L5,10:00,Friday,2021-07-01T16:28:56Z,36886778,...,2021-10-11T23:52:11Z,30,12:00,Null,2021-10-20T08:18:53Z,afrakero,Alvan Frake,ML/AI - SageMaker,1,2022-05-02T03:46:12Z
997,51097937,Ignacio Chivers,40720089,Security Engineer,Internship,L9,11:00,Monday,2022-04-22T09:03:40Z,51097937,...,2022-05-22T23:47:06Z,18,12:00,Null,2021-06-30T17:00:33Z,ttilleyrp,Torry Tilley,Compute and HPC - EC2 (Elastic Compute Cloud),4,2022-05-24T05:06:30Z
998,88855859,Stillman Sproson,74769886,Software Engineer,Contract,L8,13:00,Friday,2022-05-19T20:51:46Z,88855859,...,2021-08-31T11:48:22Z,33,14:00,Null,2022-03-07T12:05:38Z,bwadwellrq,Bryn Wadwell,Analytics - Lake Formation,4,2022-06-11T22:42:18Z


Feature Engineering:

In [41]:
#master_path = 's3://ml-analytics-data-2022/training-new/master-full-dataset.csv'
master_df_new = pd.read_csv("s3://hiring-curated-bucket/master-table/test_csv/date_col=2022-07-28-05-06-21.423/*")
master_df_new

,hircand_Interview_ID,hircand_Candidate_Name,hircand_Candidate_ICMS_ID,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hircand_Last_Updated,hiremp_Interview_ID,...,sched_timestamp,sched_week_number,sched_slot_hour,sched_modified,sched_last_updated,emp_alias,emp_name,emp_skill,emp_skill_rating,emp_last_updated
0,51092998,Agnese Normanton,7203708,EC2 Systems Engineer,Contract,L9,15:00,Tuesday,2022-02-04T16:18:37Z,51092998,...,2021-12-08T20:21:20Z,4,16:00,Null,2022-04-26T10:54:21Z,ldelepine0,Linea Delepine,Compute and HPC - EC2 (Elastic Compute Cloud),1,2021-12-27T11:35:02Z
1,99241636,Davey Abrams,15918070,Professional Services Consultant,Contract,L8,13:00,Monday,2022-03-20T01:04:55Z,99241636,...,2021-08-18T07:24:37Z,49,12:00,Null,2022-04-24T09:18:34Z,aharral1,Arielle Harral,Database - RDS SQL Server,5,2021-12-29T20:03:24Z
2,64517874,Marrissa Langhor,67232318,EC2 Systems Engineer,Part-time,L6,14:00,Thursday,2022-03-18T12:56:21Z,64517874,...,2021-11-03T01:47:35Z,3,12:00,Null,2021-08-22T12:22:13Z,sdryburgh2,Shayne Dryburgh,ML/AI - Rekognition,1,2021-06-25T07:12:05Z
3,75309134,Aurora Ireland,82904801,Machine Learning Engineer,Internship,L6,12:00,Tuesday,2021-11-26T16:06:23Z,75309134,...,2022-01-06T09:58:41Z,23,10:00,Null,2022-06-14T09:56:02Z,chugonet3,Claudetta Hugonet,Analytics - Lake Formation,3,2021-12-03T04:03:39Z
4,31410407,Debora Greenall,76196522,Cloud Architect,Full-time,L6,16:00,Wednesday,2022-02-20T12:24:35Z,31410407,...,2022-01-31T02:03:52Z,3,14:00,Null,2022-05-29T10:38:42Z,cfawthrop4,Cyrille Fawthrop,ML/AI - Forecast,1,2022-03-20T08:45:19Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,Burgess Baumer,39023047,Delivery Practice Manager,Part-time,L5,12:00,Monday,2022-05-08T22:34:48Z,79716724,...,2021-08-28T08:13:52Z,24,17:00,Null,2021-07-10T19:16:21Z,tvedntyevrn,Terrie Vedntyev,ML/AI - Forecast,5,2022-04-29T00:23:01Z
996,36886778,Jocelyn Howman,40070522,Marketing Manager,Full-time,L5,10:00,Friday,2021-07-01T16:28:56Z,36886778,...,2021-10-11T23:52:11Z,30,12:00,Null,2021-10-20T08:18:53Z,afrakero,Alvan Frake,ML/AI - SageMaker,1,2022-05-02T03:46:12Z
997,51097937,Ignacio Chivers,40720089,Security Engineer,Internship,L9,11:00,Monday,2022-04-22T09:03:40Z,51097937,...,2022-05-22T23:47:06Z,18,12:00,Null,2021-06-30T17:00:33Z,ttilleyrp,Torry Tilley,Compute and HPC - EC2 (Elastic Compute Cloud),4,2022-05-24T05:06:30Z
998,88855859,Stillman Sproson,74769886,Software Engineer,Contract,L8,13:00,Friday,2022-05-19T20:51:46Z,88855859,...,2021-08-31T11:48:22Z,33,14:00,Null,2022-03-07T12:05:38Z,bwadwellrq,Bryn Wadwell,Analytics - Lake Formation,4,2022-06-11T22:42:18Z


In [42]:
train_path = 's3://ml-analytics-data-2022/training-new/output_1659045804/training-dataset.csv'
train_df = pd.read_csv(train_path)
train_df

,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hiremp_Employee_Alias,hiremp_Interviewer_Name,hiremp_Role,hiremp_Type,hiremp_Level,hiremp_Preferred_Time,hiremp_Preferred_Day,emp_alias,emp_name,emp_skill,emp_skill_rating,Accepted_or_Not?
0,EC2 Systems Engineer,Contract,L9,15:00,Tuesday,ldelepine0,Linea Delepine,Delivery Practice Manager,Contract,L4,10:00,Thursday,ldelepine0,Linea Delepine,Compute and HPC - EC2 (Elastic Compute Cloud),1,Yes
1,Professional Services Consultant,Contract,L8,13:00,Monday,aharral1,Arielle Harral,Software Development Engineer,Part-time,L7,14:00,Wednesday,aharral1,Arielle Harral,Database - RDS SQL Server,5,No
2,EC2 Systems Engineer,Part-time,L6,14:00,Thursday,sdryburgh2,Shayne Dryburgh,Marketing Manager,Full-time,L6,14:00,Thursday,sdryburgh2,Shayne Dryburgh,ML/AI - Rekognition,1,No
3,Machine Learning Engineer,Internship,L6,12:00,Tuesday,chugonet3,Claudetta Hugonet,Security Engineer,Contract,L9,16:00,Monday,chugonet3,Claudetta Hugonet,Analytics - Lake Formation,3,Yes
4,Cloud Architect,Full-time,L6,16:00,Wednesday,cfawthrop4,Cyrille Fawthrop,Cloud Support Engineer,Part-time,L5,12:00,Friday,cfawthrop4,Cyrille Fawthrop,ML/AI - Forecast,1,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Machine Learning Engineer,Internship,L7,15:00,Friday,mmerittrm,Marja Meritt,Professional Services Consultant,Internship,L8,15:00,Wednesday,mmerittrm,Marja Meritt,Analytics - Lake Formation,1,No
796,Marketing Manager,Full-time,L5,10:00,Friday,afrakero,Alvan Frake,Solutions Architect,Internship,L8,14:00,Wednesday,afrakero,Alvan Frake,ML/AI - SageMaker,1,Yes
797,Security Engineer,Internship,L9,11:00,Monday,ttilleyrp,Torry Tilley,Product Manager,Contract,L4,14:00,Tuesday,ttilleyrp,Torry Tilley,Compute and HPC - EC2 (Elastic Compute Cloud),4,Yes
798,Software Engineer,Contract,L8,13:00,Friday,bwadwellrq,Bryn Wadwell,EC2 Systems Engineer,Part-time,L9,14:00,Wednesday,bwadwellrq,Bryn Wadwell,Analytics - Lake Formation,4,Yes


In [43]:
test_path = 's3://ml-analytics-data-2022/training-new/output_1659045962/testing-dataset.csv'
test_df = pd.read_csv(test_path)
test_df

,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hiremp_Employee_Alias,hiremp_Interviewer_Name,hiremp_Role,hiremp_Type,hiremp_Level,hiremp_Preferred_Time,hiremp_Preferred_Day,emp_alias,emp_name,emp_skill,emp_skill_rating,Accepted_or_Not?
0,Data Scientist,Internship,L6,13:00,Friday,hlowles7,Hatti Lowles,Program Manager,Internship,L4,9:00,Wednesday,hlowles7,Hatti Lowles,Compute and HPC - EC2 (Elastic Compute Cloud),5,Yes
1,Technical Account Manager,Contract,L9,12:00,Friday,sdelmontib,Stacia Delmonti,Machine Learning Engineer,Full-time,L8,13:00,Tuesday,sdelmontib,Stacia Delmonti,ML/AI - Forecast,5,No
2,Data Scientist,Full-time,L5,8:00,Thursday,bgreshamh,Bronson Gresham,Delivery Practice Manager,Part-time,L4,15:00,Friday,bgreshamh,Bronson Gresham,Analytics - Lake Formation,1,Yes
3,Software Engineer,Internship,L4,8:00,Monday,zellingsi,Zach Ellings,Data Engineer,Contract,L6,15:00,Thursday,zellingsi,Zach Ellings,Database - RDS SQL Server,1,No
4,Cloud Architect,Full-time,L6,16:00,Thursday,cingyont,Carlina Ingyon,Supply Chain Manager,Part-time,L6,16:00,Thursday,cingyont,Carlina Ingyon,Database - RDS SQL Server,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Solutions Architect,Part-time,L9,15:00,Thursday,eplimmerr7,Ernestus Plimmer,Data Scientist,Contract,L6,17:00,Monday,eplimmerr7,Ernestus Plimmer,Mobile - Pinpoint,2,No
196,Cloud Architect,Contract,L7,12:00,Wednesday,aduddellra,Agnola Duddell,Financial Analyst,Contract,L9,17:00,Monday,aduddellra,Agnola Duddell,Database - RDS SQL Server,1,Yes
197,Cloud Architect,Contract,L6,10:00,Friday,rsticklesrd,Raphaela Stickles,Software Development Engineer,Internship,L4,13:00,Wednesday,rsticklesrd,Raphaela Stickles,Analytics - Lake Formation,1,Yes
198,Professional Services Consultant,Full-time,L5,16:00,Tuesday,afridayrg,Agathe Friday,Product Manager,Full-time,L9,8:00,Wednesday,afridayrg,Agathe Friday,Analytics - Lake Formation,1,No


In [44]:
train_encode_path = 's3://ml-analytics-data-2022/training-new/output_1659050090/training-encoded.csv'
train_encode_df = pd.read_csv(train_encode_path)
train_encode_df

,emp_skill_rating,Accepted_or_Not?,hircand_Role_Cloud Architect,hircand_Role_Data Architect,hircand_Role_Technical Account Manager,hircand_Role_Data Engineer,hircand_Role_Cloud Support Engineer,hircand_Role_EC2 Systems Engineer,hircand_Role_Machine Learning Engineer,hircand_Role_Product Manager,...,hiremp_Preferred_Day_Thursday,hiremp_Preferred_Day_Tuesday,emp_skill_Analytics - Lake Formation,emp_skill_ML/AI - Rekognition,emp_skill_Database - RDS SQL Server,emp_skill_ML/AI - Forecast,emp_skill_ML/AI - SageMaker,emp_skill_Mobile - Pinpoint,emp_skill_Compute and HPC - EC2 (Elastic Compute Cloud),emp_skill_Analytics - Kinesis
0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
797,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
798,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
test_encode_path = 's3://ml-analytics-data-2022/training-new/output_1659050187/testing_encoded.csv'
test_encode_df = pd.read_csv(test_encode_path)
test_encode_df

,emp_skill_rating,Accepted_or_Not?,hircand_Role_Cloud Architect,hircand_Role_Data Architect,hircand_Role_Technical Account Manager,hircand_Role_Data Engineer,hircand_Role_Cloud Support Engineer,hircand_Role_EC2 Systems Engineer,hircand_Role_Machine Learning Engineer,hircand_Role_Product Manager,...,hiremp_Preferred_Day_Thursday,hiremp_Preferred_Day_Tuesday,emp_skill_Analytics - Lake Formation,emp_skill_ML/AI - Rekognition,emp_skill_Database - RDS SQL Server,emp_skill_ML/AI - Forecast,emp_skill_ML/AI - SageMaker,emp_skill_Mobile - Pinpoint,emp_skill_Compute and HPC - EC2 (Elastic Compute Cloud),emp_skill_Analytics - Kinesis
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
196,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
197,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# input
x_train = train_encode_df.drop(['Accepted_or_Not?'], axis=1)
x_test = test_encode_df.drop(['Accepted_or_Not?'], axis=1)

# output
y_train = train_encode_df.iloc[:, 1].values
y_test = test_encode_df.iloc[:, 1].values

In [47]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.20, random_state = 0)

In [48]:
x_train.shape

(800, 101)

In [49]:
(y_test)

array([0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.])

In [50]:

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [51]:
train_acc = classifier.score(x_train, y_train)
print("The Accuracy for the Training Set is {}".format(train_acc*100)+"%")

The Accuracy for the Training Set is 73.625%


In [52]:
y_pred = classifier.predict(x_test)
y_pred

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [53]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
 
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[126  19]
 [ 47   8]]


In [54]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.67


In [55]:
acc_prob = classifier.predict_proba(x_test)[:,1]
acc_prob

array([0.31330174, 0.29088394, 0.16811947, 0.65706148, 0.14910208,
       0.26297457, 0.38264311, 0.17941421, 0.24875609, 0.25237053,
       0.48929878, 0.35760667, 0.16600182, 0.25813865, 0.12520946,
       0.3518076 , 0.08692404, 0.36518676, 0.18184184, 0.18949176,
       0.2780003 , 0.52483248, 0.44029798, 0.73216855, 0.25975657,
       0.13633681, 0.20494511, 0.54955055, 0.2931416 , 0.20914934,
       0.1815125 , 0.18795952, 0.26588911, 0.53151089, 0.09675795,
       0.17811677, 0.22308794, 0.34209553, 0.32170766, 0.35715501,
       0.43055522, 0.47657983, 0.12499297, 0.14928433, 0.4918762 ,
       0.30766366, 0.39632929, 0.47472599, 0.22319781, 0.46826697,
       0.14068576, 0.28060237, 0.22056664, 0.1915625 , 0.5979859 ,
       0.24432261, 0.08599932, 0.22779106, 0.41061219, 0.17055845,
       0.31443632, 0.56712673, 0.21409373, 0.26937334, 0.25236602,
       0.05083341, 0.76247143, 0.26735113, 0.06360237, 0.49182301,
       0.11799794, 0.39228169, 0.26562651, 0.09321103, 0.55921

In [23]:
a

,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hiremp_Employee_Alias,hiremp_Interviewer_Name,hiremp_Role,hiremp_Type,hiremp_Level,hiremp_Preferred_Time,hiremp_Preferred_Day,emp_alias,emp_name,emp_skill,emp_skill_rating,Accepted_or_Not?,Predicted_Accepted_or_Not?,Probability_of_Acceptance
0,Data Scientist,Internship,L6,13:00,Friday,hlowles7,Hatti Lowles,Program Manager,Internship,L4,9:00,Wednesday,hlowles7,Hatti Lowles,Compute and HPC - EC2 (Elastic Compute Cloud),5,Yes,No,0.313302
1,Technical Account Manager,Contract,L9,12:00,Friday,sdelmontib,Stacia Delmonti,Machine Learning Engineer,Full-time,L8,13:00,Tuesday,sdelmontib,Stacia Delmonti,ML/AI - Forecast,5,No,No,0.290884
2,Data Scientist,Full-time,L5,8:00,Thursday,bgreshamh,Bronson Gresham,Delivery Practice Manager,Part-time,L4,15:00,Friday,bgreshamh,Bronson Gresham,Analytics - Lake Formation,1,Yes,No,0.168119
3,Software Engineer,Internship,L4,8:00,Monday,zellingsi,Zach Ellings,Data Engineer,Contract,L6,15:00,Thursday,zellingsi,Zach Ellings,Database - RDS SQL Server,1,No,Yes,0.657061
4,Cloud Architect,Full-time,L6,16:00,Thursday,cingyont,Carlina Ingyon,Supply Chain Manager,Part-time,L6,16:00,Thursday,cingyont,Carlina Ingyon,Database - RDS SQL Server,4,No,No,0.149102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Solutions Architect,Part-time,L9,15:00,Thursday,eplimmerr7,Ernestus Plimmer,Data Scientist,Contract,L6,17:00,Monday,eplimmerr7,Ernestus Plimmer,Mobile - Pinpoint,2,No,No,0.195926
196,Cloud Architect,Contract,L7,12:00,Wednesday,aduddellra,Agnola Duddell,Financial Analyst,Contract,L9,17:00,Monday,aduddellra,Agnola Duddell,Database - RDS SQL Server,1,Yes,No,0.434286
197,Cloud Architect,Contract,L6,10:00,Friday,rsticklesrd,Raphaela Stickles,Software Development Engineer,Internship,L4,13:00,Wednesday,rsticklesrd,Raphaela Stickles,Analytics - Lake Formation,1,Yes,No,0.469756
198,Professional Services Consultant,Full-time,L5,16:00,Tuesday,afridayrg,Agathe Friday,Product Manager,Full-time,L9,8:00,Wednesday,afridayrg,Agathe Friday,Analytics - Lake Formation,1,No,No,0.275544


In [34]:
# import sagemaker
# input_data_path = 's3://ml-analytics-data-2022/training-new/output_1659045962/testing-dataset.csv'
# output_data_path = 's3://ml-analytics-data-2022/Notebook-instance/'
# transform_job = sagemaker.transformer.Transformer(
#     model_name = classifier,
#     instance_count = 1,
#     instance_type = 'ml.m4.xlarge',
#     strategy = 'SingleRecord',
#     assemble_with = 'Line',
#     output_path = output_data_path,
#     base_transform_job_name='inference-pipelines-batch',
#     sagemaker_session=sagemaker.Session(),
#     accept = 'application/csv')
# transform_job.transform(data = input_data_path, 
#                         content_type = 'application/csv', 
#                         split_type = 'Line')

In [35]:
# from sklearn import svm
# from sklearn import datasets
# clf = svm.SVC()
# X, y= datasets.load_iris(return_X_y=True)
# clf.fit(X, y)


# import pickle
# s = pickle.dumps(clf)
# clf2 = pickle.loads(s)
# clf2.predict(X[0:1])

In [36]:
# from joblib import dump, load
# dump(clf, 'Logistic_Regression.joblib') 

In [37]:
# clf = load('Logistic_Regression.joblib')
# clf

In [61]:
test_df.to_csv(r's3://ml-analytics-data-2022/prediction-model-output/static-output/pred-output.csv', index=False, header=True)